In [2]:
import os
from collections import Counter

In [3]:
def count_objects_from_folder(path):
    counts = Counter()
    for file in os.listdir(path):
        if file.endswith(".txt"):
            with open(os.path.join(path, file)) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        obj = parts[-2]  # second last is object label
                        counts[obj] += 1
    return counts

In [4]:
def count_objects_from_folder_dior(path):
    counts = Counter()
    for file in os.listdir(path):
        if file.endswith(".txt"):
            with open(os.path.join(path, file)) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 2:
                        obj = parts[0]  # first is object label
                        counts[obj] += 1
    return counts

In [5]:
def combine_counts(path1, path2):
    total_counts = count_objects_from_folder(path1) + count_objects_from_folder(path2)
    return total_counts

In [7]:
path1 = "D:\\stuff\\datasets\\FAIR1M\\fair1m_date_1922\\val\\labelTxt"
path2 = "D:\\stuff\\datasets\\FAIR1M\\fair1m_date_1922\\train\\labelTxt"

cnt = 0
counts = combine_counts(path1, path2)
for obj, count in counts.most_common():
    cnt += count
    print(f"{obj}: {count}")
print(f"\nTOTAL FAIR1M: {cnt}")

Small-Car: 206491
Van: 200106
Dump-Truck: 33868
Dry-Cargo-Ship: 16879
Cargo-Truck: 16259
Motorboat: 15347
other-airplane: 14305
Intersection: 10156
Fishing-Boat: 8930
A220: 8154
other-vehicle: 6445
Boeing737: 5881
Tennis-Court: 4219
other-ship: 4215
Engineering-Ship: 3777
Liquid-Cargo-Ship: 3731
A321: 3611
Boeing747: 2476
Boeing787: 2300
A330: 2134
Passenger-Ship: 1858
Tugboat: 1855
Boeing777: 1780
Bus: 1673
Basketball-Court: 1666
Bridge: 1621
Baseball-Field: 1403
A350: 1396
Truck-Tractor: 1348
Excavator: 1280
Trailer: 1183
Football-Field: 1181
Warship: 1134
Roundabout: 833
ARJ21: 304
Tractor: 298
C919: 158

TOTAL FAIR1M: 590255


In [8]:
path3 = "D:\\stuff\\datasets\\DIOR\\archive\\labels"

cnt = 0
counts = count_objects_from_folder_dior(path3)
for obj, count in counts.most_common():
    cnt += count
    print(f"{obj}: {count}")
print(f"\nTOTAL DIOR: {cnt}")

7: 62400
5: 40370
0: 26414
2: 12266
8: 10104
1: 5817
6: 5509
4: 5363
9: 3967
3: 3225
10: 3114
14: 3038
16: 2165
13: 1681
18: 1327
11: 1298
17: 1268
19: 1086
15: 1049
12: 1011

TOTAL DIOR: 192472


In [9]:
path1 = "D:\\stuff\\datasets\\DOTAv2\\DOTAv2\\labels\\DOTA-v2.0_val (original format)"
path2 = "D:\\stuff\\datasets\\DOTAv2\\DOTAv2\\labels\\DOTA-v2.0_train (original format)"

cnt = 0
counts = combine_counts(path1, path2)
for obj, count in counts.most_common():
    cnt += count
    print(f"{obj}: {count}")
print(f"\nTOTAL DOTAv2: {cnt}")

small-vehicle: 219330
ship: 54353
large-vehicle: 29941
plane: 11365
storage-tank: 10617
harbor: 8902
tennis-court: 3560
swimming-pool: 3230
bridge: 3043
baseball-diamond: 965
roundabout: 885
helicopter: 730
basketball-court: 711
ground-track-field: 684
soccer-ball-field: 573
airport: 410
container-crane: 270
helipad: 106

TOTAL DOTAv2: 349675
